In [103]:
# Import dependencies
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime as dt
import sqlalchemy

### Extract the Data 

Make an API call to the database to fetch the data for February 2023.

In [2]:
# Define the URL for the database
URL = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"

In [114]:
# Define the queries to make a call for only the data for February 2023
limit = "&$limit=10000"
query = "&$where=crash_date >= '2023-02-01T00:00:00.000' AND crash_date < '2023-03-01T00:00:00.000'"
query_url = URL+limit+query
response = requests.get(query_url)
data = response.json()

In [115]:
# Check data type
type(data)

list

In [116]:
# Check the limit was enough to extract all data points
len(data)

6824

In [118]:
## Pretty print a random data point
pprint(data[8])

{'collision_id': '4602452',
 'contributing_factor_vehicle_1': 'Driver Inattention/Distraction',
 'contributing_factor_vehicle_2': 'Unspecified',
 'crash_date': '2023-02-01T00:00:00.000',
 'crash_time': '0:55',
 'latitude': '40.74478',
 'location': {'human_address': '{"address": "", "city": "", "state": "", '
                               '"zip": ""}',
              'latitude': '40.74478',
              'longitude': '-73.93262'},
 'longitude': '-73.93262',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '0',
 'number_of_motorist_killed': '0',
 'number_of_pedestrians_injured': '0',
 'number_of_pedestrians_killed': '0',
 'number_of_persons_injured': '0',
 'number_of_persons_killed': '0',
 'off_street_name': '32 PLACE',
 'on_street_name': 'QUEENS BOULEVARD',
 'vehicle_type_code1': 'Sedan',
 'vehicle_type_code2': 'Pick-up Truck'}


In [119]:
# Convert the data to a Pandas dataframe
df = pd.DataFrame(data)
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01T00:00:00.000,12:30,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01T00:00:00.000,18:34,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01T00:00:00.000,11:00,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01T00:00:00.000,18:15,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01T00:00:00.000,9:44,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

Transform the data set to required data types, and add and drop columns as necessary.

In [121]:
# Check the data types for the different columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6824 entries, 0 to 6823
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6824 non-null   object
 1   crash_time                     6824 non-null   object
 2   borough                        4481 non-null   object
 3   zip_code                       4480 non-null   object
 4   latitude                       6356 non-null   object
 5   longitude                      6356 non-null   object
 6   location                       6356 non-null   object
 7   cross_street_name              1834 non-null   object
 8   number_of_persons_injured      6824 non-null   object
 9   number_of_persons_killed       6824 non-null   object
 10  number_of_pedestrians_injured  6824 non-null   object
 11  number_of_pedestrians_killed   6824 non-null   object
 12  number_of_cyclist_injured      6824 non-null   object
 13  num

In [122]:
# Drop any rows which do not have any vehicle associated with them
df_dropped = df[df['vehicle_type_code1'].notna()]
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6721 entries, 0 to 6823
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6721 non-null   object
 1   crash_time                     6721 non-null   object
 2   borough                        4398 non-null   object
 3   zip_code                       4397 non-null   object
 4   latitude                       6254 non-null   object
 5   longitude                      6254 non-null   object
 6   location                       6254 non-null   object
 7   cross_street_name              1814 non-null   object
 8   number_of_persons_injured      6721 non-null   object
 9   number_of_persons_killed       6721 non-null   object
 10  number_of_pedestrians_injured  6721 non-null   object
 11  number_of_pedestrians_killed   6721 non-null   object
 12  number_of_cyclist_injured      6721 non-null   object
 13  num

In [123]:
# Convert the 'crash_date' column to datetime data type
df_dropped['crash_date']= pd.to_datetime(df_dropped['crash_date'])
df_dropped.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,12:30,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,18:34,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,11:00,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,18:15,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01,9:44,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# Convert the 'crash_time' column to datetime data type and keep only the hour
df_dropped['crash_time'] = pd.to_datetime(df_dropped['crash_time'], format='%H:%M').dt.hour
df_dropped.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,12,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,18,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,11,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,18,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01,9,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
# Convert the various numbers of injuries/deaths columns to integer types
columns_to_int = ['number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
                  'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed']
df_dropped[columns_to_int] = df_dropped[columns_to_int].astype(int)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6721 entries, 0 to 6823
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6721 non-null   datetime64[ns]
 1   crash_time                     6721 non-null   int64         
 2   borough                        4398 non-null   object        
 3   zip_code                       4397 non-null   object        
 4   latitude                       6254 non-null   object        
 5   longitude                      6254 non-null   object        
 6   location                       6254 non-null   object        
 7   cross_street_name              1814 non-null   object        
 8   number_of_persons_injured      6721 non-null   int64         
 9   number_of_persons_killed       6721 non-null   int64         
 10  number_of_pedestrians_injured  6721 non-null   int64         
 11  number_of_pedestr

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [126]:
# Convert lat and lon columns to float data type
df_dropped[['latitude','longitude']] = df_dropped[['latitude','longitude']].astype(float)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6721 entries, 0 to 6823
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6721 non-null   datetime64[ns]
 1   crash_time                     6721 non-null   int64         
 2   borough                        4398 non-null   object        
 3   zip_code                       4397 non-null   object        
 4   latitude                       6254 non-null   float64       
 5   longitude                      6254 non-null   float64       
 6   location                       6254 non-null   object        
 7   cross_street_name              1814 non-null   object        
 8   number_of_persons_injured      6721 non-null   int64         
 9   number_of_persons_killed       6721 non-null   int64         
 10  number_of_pedestrians_injured  6721 non-null   int64         
 11  number_of_pedestr

In [127]:
# Bin the rows based on the hour when the incident happened
df_binned = df_dropped
bins = [0,4,8,12,16,20,24]
labels = ["12AM - 4AM","4AM - 8AM","8AM - 12PM","12PM - 4PM","4PM - 8PM","8PM - 12AM"]
df_binned['period_of_day'] = pd.cut(x = df_binned['crash_time'], bins = bins, labels = labels)
df_binned.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5,period_of_day
0,2023-02-01,12,BROOKLYN,11215,40.666996,-73.992300,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM
1,2023-02-01,18,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
2,2023-02-01,11,BROOKLYN,11215,40.672160,-73.987210,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM
3,2023-02-01,18,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
4,2023-02-01,9,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM


In [128]:
# Create a new database and keep only the columns needed
df_final = df_binned[['collision_id','crash_date','crash_time','borough','latitude','longitude','number_of_persons_injured', 'number_of_persons_killed', 
                      'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 
                      'number_of_motorist_injured', 'number_of_motorist_killed', 'vehicle_type_code1', 'vehicle_type_code2', 'vehicle_type_code_3',
                      'vehicle_type_code_4','vehicle_type_code_5']]
df_final.head()

,collision_id,crash_date,crash_time,borough,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,4603283,2023-02-01,12,BROOKLYN,40.666996,-73.992300,0,0,0,0,0,0,0,0,Sedan,Box Truck,NaN,NaN,NaN
1,4602509,2023-02-01,18,NaN,40.754177,-73.984604,0,0,0,0,0,0,0,0,Taxi,Sedan,NaN,NaN,NaN
2,4603101,2023-02-01,11,BROOKLYN,40.672160,-73.987210,0,0,0,0,0,0,0,0,Garbage or Refuse,NaN,NaN,NaN,NaN
3,4603312,2023-02-01,18,MANHATTAN,40.758465,-73.970924,1,0,0,0,1,0,0,0,Sedan,Bike,NaN,NaN,NaN
4,4602297,2023-02-01,9,BROOKLYN,NaN,NaN,1,0,0,0,0,0,1,0,Station Wagon/Sport Utility Vehicle,Box Truck,NaN,NaN,NaN


## Load

Load the pandas dataframe to a SQlite database.

In [129]:
# Create an engine to access the sqlite database file
engine=sqlalchemy.create_engine('sqlite:///data/db.sqlite')

In [130]:
# Convert the dataframe to a "data" table in the db.sqlite database
df_final.to_sql('data', if_exists='replace', index=False, con=engine)